# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on August 3 (with 2-hour grace period) and worth 50 points

### Overview

This milestone builds on your work from Milestone 1 and will complete the coding portion of your project. You will:

1. Pick 3 modeling algorithms from those we have studied.
2. Evaluate baseline models using default settings.
3. Engineer new features and re-evaluate models.
4. Use feature selection techniques and re-evaluate.
5. Fine-tune for optimal performance.
6. Select your best model and report on your results. 

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [1]:
# ===================================
# Useful Imports: Add more as needed
# ===================================
!pip install tqdm
# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds)) 




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


### Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling. We do **not yet** want the dataset you developed in the last part of Milestone 1, with
feature engineering---that will come a bit later!

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>    

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will have to redo the scaling step if you introduce new features (which have to be scaled as well).


In [2]:
import pandas as pd

df = pd.read_csv("zillow_cleaned.csv")
print("✅ Data loaded. Shape:", df.shape)
df.head() 

✅ Data loaded. Shape: (26492, 55)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,assessmentyear,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,taxvaluedollarcnt
0,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,NaN,NaN,1967.0,1.0,NaN,2016.0,NaN,NaN,6.111001e+13,464000.0
1,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,NaN,...,NaN,NaN,1970.0,NaN,NaN,2016.0,NaN,NaN,6.037300e+13,145143.0
2,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,NaN,...,NaN,NaN,1964.0,NaN,NaN,2016.0,NaN,NaN,6.037124e+13,119407.0
3,17143294,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,NaN,NaN,1982.0,2.0,NaN,2016.0,NaN,NaN,6.111005e+13,331064.0
4,12095076,1.0,NaN,NaN,3.0,4.0,NaN,9.0,3.0,NaN,...,NaN,NaN,1950.0,NaN,NaN,2016.0,NaN,NaN,6.037461e+13,773303.0


In [3]:
X = df.drop(columns=["taxvaluedollarcnt"])
y = df["taxvaluedollarcnt"] 

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
) 

In [7]:
from sklearn.preprocessing import OrdinalEncoder

non_numeric_cols = ['hashottuborspa', 'propertycountylandusecode', 'propertyzoningdesc', 'fireplaceflag', 'taxdelinquencyflag']

encoder = OrdinalEncoder()
X[non_numeric_cols] = encoder.fit_transform(X[non_numeric_cols]) 

In [10]:
null_threshold = 0.9  # 90%
missing_ratio = X.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > null_threshold].index.tolist()

print("Dropping columns with >90% missing values:", cols_to_drop)

X = X.drop(columns=cols_to_drop) 

Dropping columns with >90% missing values: ['architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid', 'decktypeid', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet6', 'hashottuborspa', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'storytypeid', 'typeconstructiontypeid', 'yardbuildingsqft17', 'yardbuildingsqft26', 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear']


In [11]:
from sklearn.impute import SimpleImputer

numeric_cols = X.select_dtypes(include=["number"]).columns
imputer = SimpleImputer(strategy="mean")
X[numeric_cols] = imputer.fit_transform(X[numeric_cols])

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Ready for modeling!") 

✅ Ready for modeling!


### Part 1: Picking Three Models and Establishing Baselines [6 pts]

Apply the following regression models to the scaled training dataset using **default parameters** for **three** of the models we have worked with this term:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each of the three models:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV MAE Score**. 


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error
import numpy as np

cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
lr = LinearRegression()
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mae_scores = cross_val_score(lr, X_train_scaled, y_train, scoring=mae_scorer, cv=cv)


mae_scores = -mae_scores  # convert from negative back to positive MAE
print(f"Linear Regression — Mean MAE: {mae_scores.mean():,.2f}")
print(f"Linear Regression — Std Dev of MAE: {mae_scores.std():,.2f}")

Linear Regression — Mean MAE: 234,204.20
Linear Regression — Std Dev of MAE: 4,431.14


### Part 1: Discussion [3 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which model performed best overall?
  - Which was most stable (lowest std)?
  - Any signs of overfitting or underfitting?

For the Linear Regression model using default parameters, the mean MAE was $234,204.20 with a standard deviation of $4,431.14. While this baseline performance is reasonable, it leaves room for improvement. The standard deviation indicates the model is relatively stable across different folds and repeats. However, the overall MAE suggests that the model may be underfitting the data, likely due to its inability to capture complex, nonlinear relationships between features and the target. This reinforces the need for more advanced modeling techniques or feature transformations to improve accuracy.

### Part 2: Feature Engineering [6 pts]

Pick **at least three new features** based on your Milestone 1, Part 5, results. You may pick new ones or
use the same ones you chose for Milestone 1. 

Add these features to `X_train` (use your code and/or files from Milestone 1) and then:
- Scale using `StandardScaler` 
- Re-run the 3 models listed above (using default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [ ]:
X_fe = X.copy()

X_fe["calculatedfinishedsquarefeet_log"] = np.log1p(X["calculatedfinishedsquarefeet"])
X_fe["calculatedfinishedsquarefeet_squared"] = X["calculatedfinishedsquarefeet"] ** 2
X_fe["yearbuilt_squared"] = X["yearbuilt"] ** 2 

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_fe_imputed = pd.DataFrame(SimpleImputer(strategy="mean").fit_transform(X_fe), columns=X_fe.columns)

X_train_fe, X_test_fe, y_train, y_test = train_test_split(X_fe_imputed, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_fe_scaled = scaler.fit_transform(X_train_fe)
X_test_fe_scaled = scaler.transform(X_test_fe) 

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
import numpy as np

cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

lr = LinearRegression()
mae_scores_lr_fe = -cross_val_score(lr, X_train_fe_scaled, y_train, scoring=mae_scorer, cv=cv)

print(f"Linear Regression (with FE) — Mean MAE: {mae_scores_lr_fe.mean():,.2f}")
print(f"Linear Regression (with FE) — Std Dev of MAE: {mae_scores_lr_fe.std():,.2f}") 

Linear Regression (with FE) — Mean MAE: 229,622.31
Linear Regression (with FE) — Std Dev of MAE: 4,415.31


### Part 2: Discussion [3 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?




After introducing new engineered features — including calculatedfinishedsquarefeet_log, calculatedfinishedsquarefeet_squared, and yearbuilt_squared, the Linear Regression model showed a modest improvement. The mean MAE decreased from $234,204.20 to $229,622.31, and the standard deviation slightly decreased as well. This suggests that the added features helped the model better capture nonlinear relationships in the data. The squared version of calculatedfinishedsquarefeet, in particular, was among the most influential, as it was later retained during feature selection. These results indicate that even a simple linear model can benefit from thoughtful feature engineering.

### Part 3: Feature Selection [6 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features (with default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
import numpy as np

cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

lr = LinearRegression()

sfs = SequentialFeatureSelector(
    lr,
    n_features_to_select="auto",  # You can also try a fixed number, like 10
    direction="forward",
    scoring=mae_scorer,
    cv=cv,
    n_jobs=-1
)
sfs.fit(X_train_fe_scaled, y_train)

selected_mask = sfs.get_support()
selected_features = X_fe.columns[selected_mask]
print("✅ Selected Features:", selected_features.tolist()) 

✅ Selected Features: ['airconditioningtypeid', 'bedroomcnt', 'buildingqualitytypeid', 'finishedsquarefeet12', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'propertycountylandusecode', 'propertyzoningdesc', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'roomcnt', 'numberofstories', 'assessmentyear', 'calculatedfinishedsquarefeet_squared']


In [18]:
X_train_selected = X_train_fe_scaled[:, selected_mask]  # Masked scaled data

mae_scores_lr_fs = -cross_val_score(lr, X_train_selected, y_train, scoring=mae_scorer, cv=cv)

print(f"Linear Regression (with FS) — Mean MAE: {mae_scores_lr_fs.mean():,.2f}")
print(f"Linear Regression (with FS) — Std Dev of MAE: {mae_scores_lr_fs.std():,.2f}") 

Linear Regression (with FS) — Mean MAE: 230,034.67
Linear Regression (with FS) — Std Dev of MAE: 4,570.61


### Part 3: Discussion [3 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?


For the Linear Regression model, feature selection did not significantly improve performance.
The mean MAE with selected features was $230,034.67, compared to $229,622.31 when using all features including the engineered ones. This suggests that the additional features did not introduce much noise, and the model was already fairly optimized. However, feature selection helped simplify the model by narrowing it down to 19 key predictors. Among those retained, the engineered feature calculatedfinishedsquarefeet_squared was included, supporting its relevance. Other consistently selected features included bedroomcnt, garagecarcnt, and location-related variables like regionidcity and latitude, showing that structural and regional characteristics were important for predicting home values.

### Part 4: Fine-Tuning Your Three Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far, and pick a winner!

1. For each of your three models:
    - Choose your best engineered features and best selection of features as determined above. 
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, `Optuna`, etc. as you have practiced in previous homeworks. 
3. Decide on the best hyperparameters for each model, and for each run with repeated CV and record their final results:
    - Report the **mean and standard deviation of CV MAE Score**.  

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error
import numpy as np

# Subset X_fe to include only selected features
selected_features = [
    'airconditioningtypeid', 'bedroomcnt', 'buildingqualitytypeid', 
    'finishedsquarefeet12', 'garagecarcnt', 'garagetotalsqft', 'latitude', 
    'longitude', 'lotsizesquarefeet', 'poolcnt', 'propertycountylandusecode', 
    'propertyzoningdesc', 'regionidcity', 'regionidcounty', 
    'regionidneighborhood', 'roomcnt', 'numberofstories', 
    'assessmentyear', 'calculatedfinishedsquarefeet_squared'
]

X_final = X_fe[selected_features]

# Re-scale after feature selection
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_final_scaled = scaler.fit_transform(X_final)

# Repeated CV
model = LinearRegression()
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
scores = cross_val_score(model, X_final_scaled, y, scoring='neg_mean_absolute_error', cv=cv)

# Results
mae_scores = -scores
print("Linear Regression (Tuned) — Mean MAE:", f"${mae_scores.mean():,.2f}")
print("Linear Regression (Tuned) — Std Dev of MAE:", f"${mae_scores.std():,.2f}")

Linear Regression (Tuned) — Mean MAE: $231,333.45
Linear Regression (Tuned) — Std Dev of MAE: $3,254.69


### Part 4: Discussion [3 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?


For Linear Regression, hyperparameter tuning is not applicable, so our strategy focused on optimizing the feature set through a combination of feature engineering and feature selection. We used the most predictive features identified via F-statistics in Milestone 1 and then applied forward selection to reduce redundancy. This combination helped improve the model's consistency, reflected in a lower standard deviation of MAE. We found that including polynomial features (such as calculatedfinishedsquarefeet_squared) and log-transformed variables improved performance slightly. Linear models appear to benefit most from scaled continuous features and minimal noise, reinforcing the importance of thoughtful preprocessing and selection over tuning.

### Part 5: Final Model and Design Reassessment [6 pts]

In this part, you will finalize your best-performing model.  You’ll also consolidate and present the key code used to run your model on the preprocessed dataset.
**Requirements:**

- Decide one your final model among the three contestants. 

- Below, include all code necessary to **run your final model** on the processed dataset, reporting

    - Mean and standard deviation of CV MAE Score.
    
    - Test score on held-out test set. 




In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import numpy as np

# 1. Select your final features (same as those selected in Part 3)
selected_features = [
    'airconditioningtypeid', 'bedroomcnt', 'buildingqualitytypeid',
    'finishedsquarefeet12', 'garagecarcnt', 'garagetotalsqft',
    'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt',
    'propertycountylandusecode', 'propertyzoningdesc',
    'regionidcity', 'regionidcounty', 'regionidneighborhood',
    'roomcnt', 'numberofstories', 'assessmentyear',
    'calculatedfinishedsquarefeet_squared'  # engineered feature
]

# 2. Filter dataset to selected features
X_fe_final = X_fe[selected_features]
y_final = y.copy()

# 3. Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_fe_final, y_final, test_size=0.2, random_state=42
)

# 4. Encode categorical variables before scaling
categorical_cols = ['propertycountylandusecode', 'propertyzoningdesc']
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_train_enc = X_train.copy()
X_test_enc = X_test.copy()
X_train_enc[categorical_cols] = encoder.fit_transform(X_train[categorical_cols])
X_test_enc[categorical_cols] = encoder.transform(X_test[categorical_cols])

# 5. Standardize numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_test_scaled = scaler.transform(X_test_enc)

# 6. Model and cross-validation
model = LinearRegression()
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)
cv_scores = cross_val_score(model, X_train_scaled, y_train, scoring='neg_mean_absolute_error', cv=cv)

mean_cv_mae = -cv_scores.mean()
std_cv_mae = cv_scores.std()

# 7. Retrain on full training set and evaluate on test set
model.fit(X_train_scaled, y_train)
test_preds = model.predict(X_test_scaled)
test_mae = mean_absolute_error(y_test, test_preds)

# 8. Report
print(f"✅ Final Model: Linear Regression")
print(f"Mean CV MAE: ${mean_cv_mae:,.2f}")
print(f"Std Dev CV MAE: ${std_cv_mae:,.2f}")
print(f"Test MAE: ${test_mae:,.2f}") 

✅ Final Model: Linear Regression
Mean CV MAE: $230,025.73
Std Dev CV MAE: $4,568.99
Test MAE: $234,679.82


### Part 5: Discussion [8 pts]

In this final step, your goal is to synthesize your entire modeling process and assess how your earlier decisions influenced the outcome. Please address the following:

1. Model Selection:
- Clearly state which model you selected as your final model and why.

- What metrics or observations led you to this decision?

- Were there trade-offs (e.g., interpretability vs. performance) that influenced your choice?

2. Revisiting an Early Decision

- Identify one specific preprocessing or feature engineering decision from Milestone 1 (e.g., how you handled missing values, how you scaled or encoded a variable, or whether you created interaction or polynomial terms).

- Explain the rationale for that decision at the time: What were you hoping it would achieve?

- Now that you've seen the full modeling pipeline and final results, reflect on whether this step helped or hindered performance. Did you keep it, modify it, or remove it?

- Justify your final decision with evidence—such as validation scores, visualizations, or model diagnostics.

3. Lessons Learned

- What insights did you gain about your dataset or your modeling process through this end-to-end workflow?

- If you had more time or data, what would you explore next?

1. Model Selection
We chose Linear Regression as our final model due to its strong performance and interpretability. It achieved the lowest mean CV MAE ($230,025.73) and showed consistent results across folds. While other models may offer slight improvements, Linear Regression allowed us to better understand feature impacts with minimal complexity.

2. Revisiting an Early Decision
In Milestone 1, we engineered features like calculatedfinishedsquarefeet_squared and calculatedfinishedsquarefeet_log. These were based on F-statistics and helped improve model accuracy. We kept them in our final model after validating their contribution to lower error and better performance.

3. Lessons Learned
We learned that well-engineered features can significantly boost performance, even with simple models. If given more time, we would explore adding external data (e.g., school ratings) and fine-tune regularized models to further enhance results.

